In [13]:
import torchdata.datapipes as dp
from torch.utils.data import DataLoader
import torch.nn as nn
import torch
import numpy as np
import ot
import matplotlib.pyplot as plt
import timeit
import copy
from math import sqrt



In [14]:
device = torch.device('cpu')

noise_size = 62

# Number of training epochs using classical training
num_epochs = 4

# Number of iterations of Chen training
num_Chen_iters = 5000

# 'Adam' of 'RMSProp'
which_optimizer = 'Adam'

# Learning rate for optimizers
lr = 0.0001

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

ngpu = 0

# To keep the criterion Lipschitz
weight_cliping_limit = 0.01

batch_size = 1024

test_batch_size = 65536

w_dim = 5

a_dim = int(w_dim*(w_dim - 1)//2)

# if 1 use GAN1, if 2 use GAN2, etc.
which_model = 1

In [ ]:
# CHEN RELATION

def chen_combine(w_a_in: torch.TensorType):
    # the batch dimension of the inputs will be quartered
    out_size = w_a_in.size(0)//2
    assert 2*out_size == w_a_in.size(0)
    assert w_a_in.size(1) == w_dim + a_dim

    # w_0_s is from 0 to t/2 and w_s_t is from t/2 to t
    w_0_s,w_s_t = w_a_in.chunk(2)
    result = torch.clone(w_0_s + w_s_t)
    result[:,:w_dim] = sqrt(0.5)*result[:,:w_dim]
    result[:,w_dim:(w_dim+a_dim)] = 0.5*result[:,w_dim:(w_dim+a_dim)]

    idx = w_dim
    for k in range(w_dim - 1):
        for l in range(k+1,w_dim):
            correction_term = 0.25*(w_0_s[:,k]*w_s_t[:,l] - w_0_s[:,l]*w_s_t[:,k])
            result[:,idx] += correction_term
            idx += 1

    return result

# prints the 2-Wasserstein distances (in each of the Levy-area dimensions) between the input and chen_combine(chen_combine(input))

def chen_error_2step(w_a_in: torch.TensorType):
    combined_data = chen_combine(w_a_in)
    combined_data = chen_combine(combined_data)
    return [sqrt(ot.wasserstein_1d(combined_data[:,w_dim+i],w_a_in[:,w_dim+i],p=2)) for i in range(a_dim)]

In [15]:
# create dataloader for samples

def row_processer(row):
    return np.array(row, dtype= np.float32)

filename = f"samples/samples_{w_dim}-dim.csv"
datapipe = dp.iter.FileOpener([filename], mode='b')
datapipe = datapipe.parse_csv(delimiter=',')
datapipe = datapipe.map(row_processer)
dataloader = DataLoader(dataset=datapipe, batch_size=batch_size, num_workers=2)

# Check if the dimensions match
d = next(iter(dataloader))
if d.size(1) != a_dim + w_dim:
    print("!!!!!!!!!!!!!!!!!!!!!!!!! WRONG DATA DIMENSIONS !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")

In [16]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    if classname.find("BatchNorm") != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0.0)

In [17]:
# GAN 1

class Generator1(nn.Module):
    def __init__(self):
        super(Generator1, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(w_dim+noise_size,512),
            nn.BatchNorm1d(512),
            nn.ReLU(),

            nn.Linear(512,512),
            nn.BatchNorm1d(512),
            nn.ReLU(),

            nn.Linear(512,128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Linear(128,a_dim)
        )

    def forward(self, input):
        return self.main(input)


class Discriminator1(nn.Module):
    def __init__(self):
        super(Discriminator1, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(w_dim + a_dim,512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2),
            nn.Linear(512,512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2),
            nn.Linear(512,128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.2),
            nn.Linear(128,1),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [18]:
# GAN 2

class Generator2(nn.Module):
    def __init__(self):
        super(Generator2, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(w_dim+noise_size,1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),



            nn.Linear(1024,1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(),

            nn.Linear(1024,512),
            nn.BatchNorm1d(512),
            nn.ReLU(),

            nn.Linear(512,a_dim)
        )

    def forward(self, input):
        return self.main(input)



class Discriminator2(nn.Module):
    def __init__(self):
        super(Discriminator2, self).__init__()
        self.main = nn.Sequential(
            nn.Linear(w_dim + a_dim,1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.1),


            nn.Linear(1024,1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.1),

            nn.Linear(1024,256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.1),

            nn.Linear(256,1),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [19]:
# initialize nets
if which_model == 1:
    netD = Discriminator1().to(device)
    netG = Generator1().to(device)
elif which_model == 2:
    netD = Discriminator2().to(device)
    netG = Generator2().to(device)


netD.apply(weights_init)
netG.apply(weights_init)

Generator(
  (main): Sequential(
    (0): Linear(in_features=67, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=1024, out_features=1024, bias=True)
    (4): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=1024, out_features=512, bias=True)
    (7): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Linear(in_features=512, out_features=10, bias=True)
  )
)

In [20]:
# Initialise optimiser

if which_optimizer == 'Adam':
    optG = torch.optim.Adam(netG.parameters(),lr = lr, betas=(beta1,0.999))
    optD = torch.optim.Adam(netD.parameters(), lr = lr, betas=(beta1,0.999))
elif which_optimizer == 'RMSProp':
    optG = torch.optim.RMSprop(netG.parameters(), lr = lr)
    optD = torch.optim.RMSprop(netD.parameters(), lr = lr)

# A fixed W increment for testing purposes
W_fixed: torch.Tensor = torch.tensor([1.0,-0.5,-1.2,-0.3,0.7,0.2,-0.9,0.1,1.7])


W_fixed = W_fixed[:w_dim].unsqueeze(1).transpose(1,0)
W_fixed = W_fixed.expand((test_batch_size,w_dim))

# Load "true" samples generated from this fixed W increment
test_filename = f"samples/fixed_samples_{w_dim}-dim.csv"
A_fixed_true = np.genfromtxt(test_filename,dtype=float,delimiter=',',)
A_fixed_true = A_fixed_true[:,w_dim:(w_dim+a_dim)]

wass_errors = []
chen_errors = []

iters = 0

one = torch.FloatTensor([1])
mone = one * -1

In [21]:
# Early stopping setup

# Will have two backup points:
# One where the sum of Wasserstein errors was minimal and one where the max was minimal

min_sum = float('inf')
min_sum_errors = [1.0 for i in range(a_dim)]
min_sum_paramsG = copy.deepcopy(netG.state_dict())
min_sum_paramsD = copy.deepcopy(netD.state_dict())

# min_max_err = float('inf')
# min_max_errors = [1.0 for i in range(a_dim)]
# min_max_paramsG = copy.deepcopy(netG.state_dict())
# min_max_paramsD = copy.deepcopy(netD.state_dict())

In [ ]:
for epoch in range(num_epochs):

    for i, data in enumerate(dataloader):
        netD.zero_grad()

        # weight clipping so critic is lipschitz
        for p in netD.parameters():
            p.data.clamp_(-weight_cliping_limit, weight_cliping_limit)

        # check actual batch size (last batch could be shorter)
        b_size = data.size(0)

        # Train Discriminator
        # first on real data
        out_D_real = netD(data)
        lossDr = out_D_real.mean(0).view(1)
        lossDr.backward(one)

        # then on fake data

        # data has shape (b_size, w_dim + a_dim) where w_dim are the dimensions of the driving BM and a_dim is the dim of Levy Areas
        W = data[:,:w_dim]
        A_real = data[:,w_dim:(w_dim + a_dim)]
        noise = torch.randn((b_size,noise_size), dtype=torch.float, device=device)
        gen_in = torch.cat((noise,W),1)
        # generate fake data
        generated_A = netG(gen_in)
        fake_in = torch.cat((W,generated_A.detach()),1)

        lossDf = netD(fake_in)
        lossDf = lossDf.mean(0).view(1)
        lossDf.backward(mone)
        lossD = lossDr - lossDf
        optD.step()

        # train Generator with probability 1/5
        if np.random.randint(1,6) == 5:
            netG.zero_grad()

            fake_in = torch.cat((W,generated_A),1)
            lossG = netD(fake_in)
            lossG = lossG.mean(0).view(1)
            lossG.backward(one)
            optG.step()

        if iters%100 == 0:
            # Test Wasserstein error for fixed W
            noise = torch.randn((test_batch_size,noise_size), dtype=torch.float, device=device)
            g_in = torch.cat((noise,W_fixed),1)
            A_fixed_gen = netG(g_in).detach().numpy()
            errors = [sqrt(ot.wasserstein_1d(A_fixed_true[:,i],A_fixed_gen[:,i],p=2)) for i in range(a_dim)]

            # Test Chen discrepancy
            W = torch.randn((4*batch_size, w_dim), dtype= torch.float, device=device)
            noise = torch.randn((4*batch_size,noise_size), dtype=torch.float, device=device)
            gen_in = torch.cat((noise,W),1)
            A_gen = netG(gen_in)
            w_a = torch.cat((W,A_gen.detach()),1)
            ch_err = chen_error_2step(w_a)

            # Print out partial results
            pretty_errors = ["{0:0.5f}".format(i) for i in errors]
            pretty_chen_errors = ["{0:0.5f}".format(i) for i in ch_err]
            print(f"epoch: {epoch}/{num_epochs}, iter: {iters},\n errors: {pretty_errors}, \n chen errors: {pretty_chen_errors}")
            # Save for plotting
            wass_errors.append(errors)
            chen_errors.append(ch_err)

            # Early stopping checkpoint
            error_sum = sum(errors)
            if error_sum <= min_sum:
                min_sum = error_sum
                min_sum_errors = errors
                min_sum_paramsG = copy.deepcopy(netG.state_dict())
                min_sum_paramsD = copy.deepcopy(netD.state_dict())
                print("Saved parameters")

        iters += 1

epoch: 0/15, iter: 0,
 errors: ['0.14270', '0.22039', '0.08434', '0.12691', '0.24892', '0.15976', '0.14762', '0.17953', '0.14869', '0.37229']
Saved parameters
epoch: 0/15, iter: 100,
 errors: ['0.12614', '0.11775', '0.13972', '0.05907', '0.24498', '0.25003', '0.20662', '0.16803', '0.05903', '0.39907']
Saved parameters
epoch: 0/15, iter: 200,
 errors: ['0.11545', '0.17457', '0.18115', '0.17742', '0.21195', '0.25333', '0.21748', '0.19232', '0.12961', '0.33502']
epoch: 0/15, iter: 300,
 errors: ['0.12090', '0.09632', '0.16946', '0.08668', '0.26374', '0.23058', '0.21729', '0.27858', '0.09916', '0.37253']
epoch: 0/15, iter: 400,
 errors: ['0.15521', '0.10686', '0.11399', '0.12052', '0.26080', '0.18956', '0.17489', '0.18138', '0.03185', '0.36880']
Saved parameters
epoch: 0/15, iter: 500,
 errors: ['0.11224', '0.12227', '0.12311', '0.06955', '0.23459', '0.19410', '0.17166', '0.19517', '0.03523', '0.37004']
Saved parameters
epoch: 0/15, iter: 600,
 errors: ['0.13447', '0.11075', '0.08419', '0.

W_fixed = [1.0,-0.5,-1.2,-0.3,0.7,0.2,-0.9,0.1,1.7]
list_pairs(5) = [(1, 2), (1, 3), (1, 4), (1, 5), (2, 3), (2, 4), (2, 5), (3, 4), (3, 5), (4, 5)]

GAN2 best: ['0.06348', '0.25953', '0.06187', '0.11594', '0.12005', '0.11992', '0.07918', '0.15956', '0.16242', '0.01383']

In [11]:
# Return to early stopping checkpoint
if which_model == 1:
    best_netG = Generator1().to(device)
elif which_model == 2:
    best_netG = Generator2().to(device)

best_netG.load_state_dict(min_sum_paramsG)

torch.save(min_sum_paramsG, f'model_saves/GAN2_{w_dim}d_24epochs_generator.pt')
torch.save(min_sum_paramsD, f'model_saves/GAN2_{w_dim}d_24epochs_discriminator.pt')
# best_netD = Discriminator()
# best_netD.load_state_dict(min_sum_paramsD)

In [12]:
# Test Wasserstein error for fixed W
noise = torch.randn((test_batch_size,noise_size), dtype=torch.float, device=device)
g_in = torch.cat((noise,W_fixed),1)
A_fixed_gen = best_netG(g_in).detach().numpy()
errors = [sqrt(ot.wasserstein_1d(A_fixed_true[:,i],A_fixed_gen[:,i],p=2)) for i in range(a_dim)]

# Print out partial results
pretty_errors = ["{0:0.5f}".format(i) for i in errors]
print(f"best net errors: {pretty_errors}")

best net errors: ['0.06348', '0.25953', '0.06187', '0.11594', '0.12005', '0.11992', '0.07918', '0.15956', '0.16242', '0.01383']


best net errors: ['0.06348', '0.25953', '0.06187', '0.11594', '0.12005', '0.11992', '0.07918', '0.15956', '0.16242', '0.01383']

In [ ]:
# Draw errors through iterations

plt.figure(figsize=(10,5))
plt.title("2-Wasserstein distance of generated samples from the original samples for fixed W increment")
plt.plot(wass_errors)
plt.xlabel("iterations")
plt.ylabel("Wasserstein distance")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.title("2-Wasserstein distances after 2-step Chen recombinations")
plt.plot(chen_errors)
plt.xlabel("iterations")
plt.ylabel("2-Wasserstein distance")
plt.legend()
plt.show()

In [ ]:
chen_iters = 0
chen_training_wass_errors = []
chen_training_chen_errors = []
for i in range(num_Chen_iters):
    netD.zero_grad()

    # weight clipping so critic is lipschitz
    for p in netD.parameters():
        p.data.clamp_(-weight_cliping_limit, weight_cliping_limit)

    # Train Discriminator
    # generate 4*batch_size of fake data
    W = torch.randn((4*batch_size, w_dim), dtype= torch.float, device=device)
    noise = torch.randn((4*batch_size,noise_size), dtype=torch.float, device=device)
    gen_in = torch.cat((noise,W),1)
    A_gen = netG(gen_in)
    fake_in = torch.cat((W,A_gen.detach()),1)
    lossD_fake = netD(fake_in)
    lossD_fake = lossD_fake.mean(0).view(1)
    lossD_fake.backward(mone)

    # now use chen_combine to produce "true" data from the fake one
    # using chen_combine twice reduces batch dimension from 4*batch_size to batch_size
    true_data = chen_combine(fake_in.detach())
    true_data = chen_combine(true_data)
    assert true_data.size(0) == batch_size

    lossD_real = netD(true_data)
    lossD_real = 4 * lossD_real.mean(0).view(1) # multiply by 4 to counteract the 4x smaller batch
    lossD_real.backward(one)
    optD.step()

    # train Generator with probability 1/5
    if np.random.randint(1,6) == 5:
        netG.zero_grad()

        fake_in = torch.cat((W,A_gen),1)
        lossG = netD(fake_in)
        lossG = lossG.mean(0).view(1)
        lossG.backward(one)
        optG.step()

    if chen_iters%100 == 0:
        # Test Wasserstein error for fixed W
        noise = torch.randn((test_batch_size,noise_size), dtype=torch.float, device=device)
        g_in = torch.cat((noise,W_fixed),1)
        A_fixed_gen = netG(g_in).detach().numpy()
        errors = [sqrt(ot.wasserstein_1d(A_fixed_true[:,i],A_fixed_gen[:,i],p=2)) for i in range(a_dim)]
        chen_training_wass_errors.append(errors)

        # Test Chen discrepancy
        W = torch.randn((4*batch_size, w_dim), dtype= torch.float, device=device)
        noise = torch.randn((4*batch_size,noise_size), dtype=torch.float, device=device)
        gen_in = torch.cat((noise,W),1)
        A_gen = netG(gen_in)
        w_a = torch.cat((W,A_gen.detach()),1)
        ch_err = chen_error_2step(w_a)

        # Print out partial results
        pretty_errors = ["{0:0.5f}".format(i) for i in errors]
        pretty_chen_errors = ["{0:0.5f}".format(i) for i in ch_err]
        print(f"iter: {chen_iters}/{num_Chen_iters},\n errors: {pretty_errors}, \n chen errors: {pretty_chen_errors}")
        # Save for plotting
        chen_training_wass_errors.append(errors)
        chen_training_chen_errors.append(ch_err)


        # Early stopping checkpoint
        error_sum = sum(errors)
        if error_sum <= min_sum:
            min_sum = error_sum
            min_sum_errors = errors
            min_sum_paramsG = copy.deepcopy(netG.state_dict())
            min_sum_paramsD = copy.deepcopy(netD.state_dict())
            print("Saved parameters")

    chen_iters += 1

In [ ]:
# Time measurements

W_fixed: torch.Tensor = torch.tensor([1.0,-0.5,-1.2,-0.3,0.7,0.2,-0.9,0.1,1.7])
W_fixed = W_fixed[:w_dim].unsqueeze(1).transpose(1,0)
W_fixed = W_fixed.expand((test_batch_size,w_dim))
noise = torch.randn((test_batch_size,noise_size), dtype=torch.float, device=device)
g_in = torch.cat((noise,W_fixed),1)
netG.eval()
start_time = timeit.default_timer()
for i in range(100):
    A_fixed_out=netG(g_in)

elapsed = timeit.default_timer() - start_time
print(elapsed)

Takes 34.2s to generate 6553600 samples (original GAN)
Calling iterated_integrals(h = 1.0, err = 0.0005) 6553600-times takes 100.5s